<a href="https://colab.research.google.com/github/DhaiwatKabaria/NLP_19-20/blob/master/Assignment_2_17110049_Dhaiwat_Kabaria_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from nltk.tokenize import sent_tokenize,word_tokenize
from collections import Counter
import random
import numpy as np
from sklearn.model_selection import train_test_split
import io

Using TensorFlow backend.


In [16]:
#accessing  drive and opening file + decoding in read mode.
from google.colab import drive
drive.mount('/content/drive')
myfile1 = open("/content/drive/My Drive/Colab Notebooks/Assignment 2/trump.txt","r")

#Replace newline with space and convert to lower case
text=myfile1.read().replace("\n"," ")
text=text.lower()

Mounted at /content/drive


In [0]:
characters = sorted(list(set(text)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [18]:
vocab_size = len(characters)
print('Number of unique characters: ', vocab_size)
print(characters)

Number of unique characters:  65
[' ', '!', '"', '$', '%', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é', '–', '—', '‘', '’', '“', '”', '…']


In [0]:
X = []   # extracted sequences
Y = []   # the target - the follow up character
length = len(text)
seq_length = 100   #number of characters to consider before predicting the following character

In [20]:
for i in range(0, length - seq_length, 1):
    sequence = text[i:i + seq_length]
    label = text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])
    
print('Number of extracted sequences:', len(X))

Number of extracted sequences: 896170


In [0]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [22]:
X_modified.shape, Y_modified.shape

((896170, 100, 1), (896170, 65))

In [23]:
print("X[0].shape = {}, Y[0].shape = {}".format(X_modified[0].shape, Y_modified[0].shape))
print("X[0]: ", X_modified[0])
print("Y[0]: ", Y_modified[0])

X[0].shape = (100, 1), Y[0].shape = (65,)
X[0]:  [[0.75384615]
 [0.70769231]
 [0.53846154]
 [0.53846154]
 [0.50769231]
 [0.58461538]
 [0.        ]
 [0.21538462]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.16923077]
 [0.16923077]
 [0.16923077]
 [0.76923077]
 [0.58461538]
 [0.47692308]
 [0.67692308]
 [0.63076923]
 [0.        ]
 [0.84615385]
 [0.69230769]
 [0.78461538]
 [0.        ]
 [0.75384615]
 [0.69230769]
 [0.        ]
 [0.66153846]
 [0.78461538]
 [0.50769231]
 [0.58461538]
 [0.16923077]
 [0.        ]
 [0.        ]
 [0.76923077]
 [0.58461538]
 [0.47692308]
 [0.76923077]
 [0.09230769]
 [0.75384615]
 [0.        ]
 [0.75384615]
 [0.69230769]
 [0.        ]
 [0.67692308]
 [0.6       ]
 [0.50769231]
 [0.53846154]
 [0.16923077]
 [0.        ]
 [0.        ]
 [0.6       ]
 [0.75384615]
 [0.67692308]
 [0.09230769]
 [0.76923077]
 [0.        ]
 [0.58461538]
 [0.53846154]
 [0.        ]
 [0.47692308]
 [0.        ]
 [0.56923077]
 [0.73846154]
 [0.53846154]
 [0.47692308]
 [0.76923077]
 [0.        ]

In [0]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [26]:
model.fit(X_modified, Y_modified, epochs=5, batch_size=128)

Epoch 1/5
896170/896170 [==============================] - 2802s 3ms/step - loss: 2.3191
Epoch 2/5
896170/896170 [==============================] - 2791s 3ms/step - loss: 1.7962
Epoch 3/5
896170/896170 [==============================] - 2775s 3ms/step - loss: 1.6300
Epoch 4/5
118144/896170 [==>...........................] - ETA: 39:44 - loss: 1.5518

KeyboardInterrupt: ignored

In [27]:
start = np.random.randint(0, len(X)-1) # or generate random start

string_mapped = list(X[start])

full_string = [n_to_char[value] for value in string_mapped]

print("Seed:")
print("\"", ''.join(full_string), "\"")

Seed:
" nsure americans have the means to protect themselves in this age of terror. i will be always defendi "


In [0]:
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])
    
    string_mapped.append(pred_index)  
    string_mapped = string_mapped[1:len(string_mapped)] 

In [0]:
txt=""
for char in full_string:
    txt = txt+char

In [30]:
print(start)
print(txt)

208861
nsure americans have the means to protect themselves in this age of terror. i will be always defending the people of the people of the people of the people whth the people of the people of the people that we have to do it and they want to do it and they want to do it and they want to do it and they want to do it and they want to do it and they want to do it and they want to do it and they want to do it and they want to do it and they want to do it and they want to do it and they want to do it an
